 # Generate realistic handwriting from a string

## Import libraries

In [ ]:
# !pip install -r requirements.txt
# !pip install Faker # in case you want to create realistic names, places, etc.

In [ ]:
import os
os.environ["CFLAGS"] = '-std=c99' # fix for envs where there's an older version of C++ compilers, e.g. some Amazon Linux AMIs

In [ ]:
import argparse
import time
import yaml

from PIL import Image, ImageFilter, ImageOps, ImageEnhance

import numpy as np
import matplotlib.pyplot as plt

from pipeline.skeletonization import Skeletonizer
from pipeline.sampling import sample_to_penpositions
from pipeline.graves import GravesWriter
from pipeline.align import align
from pipeline.render_skeleton import render_skeleton
from pipeline.pen_style_transfer import PenStyleTransfer

from datastructures.PenPosition import plotPenPositions

Building library 'ccode' ...
running build_ext
building '__lib' extension
gcc -pthread -B /home/ec2-user/anaconda3/envs/tensorflow_p36/compiler_compat -Wl,--sysroot=/ -DNDEBUG -g -fwrapv -O3 -Wall -Wstrict-prototypes -std=c99 -fPIC -I/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/numpy/core/include -I/home/ec2-user/anaconda3/envs/tensorflow_p36/include/python3.6m -c __src/module.c -o build/temp.linux-x86_64-3.6/__src/module.o
gcc -pthread -B /home/ec2-user/anaconda3/envs/tensorflow_p36/compiler_compat -Wl,--sysroot=/ -DNDEBUG -g -fwrapv -O3 -Wall -Wstrict-prototypes -std=c99 -fPIC -I/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/numpy/core/include -I/home/ec2-user/anaconda3/envs/tensorflow_p36/include/python3.6m -c __src/compute_accelerated_stroke.c -o build/temp.linux-x86_64-3.6/__src/compute_accelerated_stroke.o
gcc -pthread -shared -B /home/ec2-user/anaconda3/envs/tensorflow_p36/compiler_compat -L/home/ec2-user/anaconda3/envs/tenso

## Helper functions

In [ ]:
class GenText:
    def __init__(self, PathToYaml):
        config = self.load_config(PathToYaml)
        self.fontStyles = self.populate_writing_styles(config['fontStyles'])
        self.penStyleTransfer = PenStyleTransfer()
        self.writer = GravesWriter()
        
    def load_config(self, PathToYaml):
        with open(PathToYaml) as f:
            return yaml.load(f, Loader=yaml.FullLoader)
    
    def get_pen_positions(self, imgPath):
        inputImg = Image.open(imgPath).convert('RGB')
        with Skeletonizer() as sk:
            skeletonBlurImg = sk.skeletonize_blurred(inputImg)
            skeletonImg = sk.skeletonize_sharp(skeletonBlurImg)
        return inputImg, sample_to_penpositions(skeletonImg)
        
    def populate_writing_styles(self, dictStyles):
        for k,v in dictStyles.items():
            image, penPosition = self.get_pen_positions(v['img_path'])
            dictStyles[k]['image'] = image
            dictStyles[k]['penPosition'] = penPosition
        return dictStyles
    
    def get_img_sizes(self, inputImg, newSkeletonBlurImg):
        orig_width, orig_height = inputImg.size
        skeleton_w, skeleton_h = newSkeletonBlurImg.size
        return {
            'orig': [orig_width, orig_height], 
            'skeleton': [skeleton_w, skeleton_h]}
    
    def resize_skeleton_image(self, imgSizes, newSkeletonBlurImg):
        orig_width, orig_height = imgSizes['orig']
        img_w, img_h = newSkeletonBlurImg.size
        background = Image.new('RGBA', 
                               (img_w, orig_height),
                               (255, 255, 255, 255),
                              )
        bg_w, bg_h = background.size
        offset = ((bg_w - img_w) // 2, (bg_h - img_h) // 2)
        background.paste(newSkeletonBlurImg, offset)
        newSkeletonBlurImg = background.convert('RGB')
        return newSkeletonBlurImg
        
    def crop_output_image(self, imgSizes, outputImg):
        orig_width, orig_height = imgSizes['orig']
        skeleton_w, skeleton_h = imgSizes['skeleton']
        left = 0
        top = (orig_height - skeleton_h)/2
        right = skeleton_w
        bottom = (orig_height + skeleton_h)/2
        outputImg = outputImg.crop((left, top, right, bottom))
        return outputImg
    
    def remove_whitespace(self,img):
        pixdata = img.load()
        width, height = img.size
        for y in range(height):
            for x in range(width):
                if pixdata[x, y] == (255, 255, 255, 255):
                    pixdata[x, y] = (255, 255, 255, 0)
        return img

    def write(self, styleNum, textOut, saveToDisk = False, showImg = False):
        fontStyle = self.fontStyles[styleNum]
        inputImg, penPositions = fontStyle['image'], fontStyle['penPosition']
        
        newPenPositions = self.writer.write(textOut, fontStyle['text'], penPositions)
        newPenPositions = align(newPenPositions, penPositions)
        newSkeletonBlurImg, newSkeletonImg = render_skeleton(newPenPositions)
        imgSizes = self.get_img_sizes(inputImg, newSkeletonBlurImg)
        newSkeletonBlurImg = self.resize_skeleton_image(imgSizes, newSkeletonBlurImg)
        
        outputImg = self.penStyleTransfer.transferStyle(newSkeletonBlurImg, inputImg)
        outputImg = self.crop_output_image(imgSizes, outputImg)
        
        if showImg:
            plt.figure('Full Pipeline', figsize=(16, 9))
            plt.subplot(3, 2, 1)
            plt.imshow(inputImg)
            plt.subplot(3, 2, 3)
            plt.imshow(inputImg)
            plt.subplot(3, 2, 5)
            plt.imshow(inputImg, cmap='binary', vmax=10)
            plotPenPositions(penPositions)
            plt.subplot(3, 2, 6)
            plt.imshow(newSkeletonImg, cmap='binary', vmax=256*10)
            plotPenPositions(newPenPositions)
            plt.subplot(3, 2, 4)
            plt.imshow(newSkeletonBlurImg)
            plt.subplot(3, 2, 2)
            plt.imshow(outputImg)
            plt.show()
            
        if saveToDisk:
            outputImg.save('output.png', 'PNG')
            
        return outputImg,newSkeletonImg,newSkeletonBlurImg

## Init class and generate text

In [ ]:
%%capture
writeText = GenText('config.yaml')

In [ ]:
stime = time.time()
im, sk_im, new_sk_im = writeText.write(1, 'Amoxidal 500mg 10 capsules')
print(f'Total time: {time.time()-stime} seconds')
im

In [ ]:
sk_im

In [ ]:
new_sk_im